# Import Packages und Functions

## Code: Breitstellen der Pakete und Funktionen
Führen Sie den unten stehnden Code aus um alle notwendigen Zusatzpakete und Funktionen verfügbar zu machen. Dieser Abschnitt muss nur ein einziges mal ausgeführt werden. 

In [ ]:
import pandas as pd # used to handle csv- or excel files as a dataframe (table object)
import numpy as np # used for basic mathematical operations
import matplotlib.pyplot as plt # package for basic data plotting
import seaborn as sns # additional package with more plotting options based on pyplot
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

# from here code for more complexe plotting functions
def _resolve_column(df: pd.DataFrame, col):
    """Resolve a column specifier for flat or MultiIndex columns.
    - col can be None, a tuple (MultiIndex), or a string.
    - Exact matches are preferred. If not found and df has MultiIndex,
      try to find columns where any level equals the string.
    """
    if col is None:
        return None, None  # (series, display_name)
    # tuple (explicit MultiIndex)
    if isinstance(col, tuple):
        if col in df.columns:
            return df[col], " - ".join(map(str, col))
        raise KeyError(f"Column tuple {col} not found in DataFrame columns.")
    # string case
    # direct exact match (flat columns)
    if col in df.columns:
        return df[col], str(col)
    # if MultiIndex, try exact match on any level
    if isinstance(df.columns, pd.MultiIndex):
        # exact level match
        matches = [c for c in df.columns if any(str(level) == col for level in c)]
        if len(matches) == 1:
            return df[matches[0]], " - ".join(map(str, matches[0]))
        if len(matches) > 1:
            # prefer a match where top-level equals col
            top_matches = [c for c in matches if str(c[0]) == col]
            chosen = top_matches[0] if top_matches else matches[0]
            print(f"Multiple columns match '{col}', using {chosen}.")
            return df[chosen], " - ".join(map(str, chosen))
        # try substring match (e.g., user passes 'PC 1' and column is ('PCA','PC 1'))
        substr_matches = [c for c in df.columns if any(col in str(level) for level in c)]
        if len(substr_matches) >= 1:
            chosen = substr_matches[0]
            print(f"No exact match for '{col}', using first substring match {chosen}.")
            return df[chosen], " - ".join(map(str, chosen))
    # fallback: try substring in flat columns
    flat_sub = [c for c in df.columns if col in str(c)]
    if len(flat_sub) >= 1:
        chosen = flat_sub[0]
        print(f"No exact match for '{col}', using first flat substring match {chosen}.")
        return df[chosen], str(chosen)
    raise KeyError(f"Column '{col}' not found in DataFrame columns.")

def plot_dataframe(df: pd.DataFrame,
                   x,
                   y,
                   z=None,
                   hue=None,
                   figsize=(9, 6),
                   palette=None,
                   point_size=40,
                   alpha=0.9,
                   cmap="viridis",
                   title=None):
    """
    Flexible plotting for DataFrame: automatic 2D/3D scatter depending on z.
    x, y, z, hue can be strings or tuples (for MultiIndex). If z is None -> 2D.
    Returns (fig, ax).
    """
    # Resolve columns and display names
    X, x_label = _resolve_column(df, x)
    Y, y_label = _resolve_column(df, y)
    Z, z_label = _resolve_column(df, z) if z is not None else (None, None)
    H, h_label = _resolve_column(df, hue) if hue is not None else (None, None)

    if X is None or Y is None:
        raise ValueError("x and y must be provided and resolvable to DataFrame columns.")

    x_vals = X.values
    y_vals = Y.values
    z_vals = Z.values if Z is not None else None
    hue_vals = H.values if H is not None else None

    is_3d = z_vals is not None

    fig = plt.figure(figsize=figsize)
    if is_3d:
        ax = fig.add_subplot(111, projection='3d')
    else:
        ax = fig.add_subplot(111)

    # No hue: single color
    if hue_vals is None:
        color = sns.color_palette()[0]
        if is_3d:
            sc = ax.scatter(x_vals, y_vals, z_vals, s=point_size, alpha=alpha, color=color)
        else:
            sc = ax.scatter(x_vals, y_vals, s=point_size, alpha=alpha, color=color)
    else:
        # Determine whether hue is numeric or categorical.
        # If dtype is object or string-like -> categorical.
        if pd.api.types.is_numeric_dtype(H):
            # continuous hue
            if is_3d:
                # 3D scatter with continuous color: map to RGBA
                norm = plt.Normalize(np.nanmin(hue_vals), np.nanmax(hue_vals))
                cmap_obj = plt.get_cmap(cmap)
                colors = cmap_obj(norm(hue_vals))
                sc = ax.scatter(x_vals, y_vals, z_vals, c=colors, s=point_size, alpha=alpha)
                # create a ScalarMappable for colorbar
                mappable = plt.cm.ScalarMappable(norm=norm, cmap=cmap_obj)
                mappable.set_array(hue_vals)
                cbar = fig.colorbar(mappable, ax=ax, pad=0.1)
                cbar.set_label(h_label or str(hue))
            else:
                sc = ax.scatter(x_vals, y_vals, c=hue_vals, cmap=cmap, s=point_size, alpha=alpha)
                cbar = fig.colorbar(sc, ax=ax, pad=0.1)
                cbar.set_label(h_label or str(hue))
        else:
            # categorical hue (strings or objects)
            categories, uniques = pd.factorize(hue_vals)
            n_cats = len(uniques)
            if palette is None:
                palette = sns.color_palette(n_colors=n_cats)
            colors = [palette[i % len(palette)] for i in categories]
            if is_3d:
                sc = ax.scatter(x_vals, y_vals, z_vals, c=colors, s=point_size, alpha=alpha)
            else:
                sc = ax.scatter(x_vals, y_vals, c=colors, s=point_size, alpha=alpha)
            # legend
            handles = []
            for i, lab in enumerate(uniques):
                handles.append(plt.Line2D([], [], marker='o', color=palette[i % len(palette)], linestyle='', markersize=6))
            ax.legend(handles, uniques, title=(h_label or str(hue)), bbox_to_anchor=(1.05, 1), loc='upper left')

    # Labels and title
    ax.set_xlabel(x_label or str(x))
    ax.set_ylabel(y_label or str(y))
    if is_3d:
        ax.set_zlabel(z_label or str(z))
    if title:
        ax.set_title(title)

    plt.tight_layout()
    plt.show()
    return fig, ax


# Daten Import

Im 1. Schritt müssen die Daten importiert werden. Hierfür wird das Datenbankpacket "pandas" verwendet. Pandas kann eine Vielzahl unterschiedliche Datentypen wie csv, excel, oder aber auch png lesen. Native Formate, die kein zusätzliches Programm benötigen, wie zum Beispiel "csv" sind für große Datensätze zu bevorzugen, da der Umweg über Excel unter Umständen sehr langsams sein kann.

## Code: Daten von csv einladen

Im Ordner "Data/CSV/" liegen zwei Datensätze vor. 

OriginalData.csv enhält grundlegende Auswertungen (Härte, E-Modul, Steifigkeit) zu den einzelnen Indents.

DataWithAdditional_Infos.csv enthält die selben Daten und noch zusätzliche Informationen über die mathemathische Beschreibung der Belastungs- und Endlastungskurve, so wie der Verläufe von Härte und Modulus über die Eindringtiefe. Diese Daten wurden durch fitting der Rohdaten gewonnen und können einen Datensatz um wertvolle Informationen erweitern. Zum Beginn können jedoch die "OriginalData.csv" eingeladen werden, um eine besser Übersichtlichkeit über die Datenstruktur zu erhalten.


In [ ]:
#=== change parameter from here ===#

file_path:str = "Data/CSV/DataWithAdditional_Infos.csv" #<-- hier muss der relative Dateipfad eingefügt werden. Beispiel "Data/CSV/MyData.csv" 

#=== dont change paramter from here ===#

# creating data frame from nanoindendation data
df = pd.read_csv(file_path, # path pointing to the file to be importe d 
                header = [0,1], # used because the dataframe has multi-index column names "("MODULUS GPa","mean") & ("MODULLUS GPa", "std")" as example
                sep = ";", # defines the separator used in csv file to separate different columns 
                decimal = "," #telling pandas the instad of "." a comma "," is used as decimal point (german number format)
                )
# Set Indent column as Index
df.index = df[("Unnamed: 0_level_0","Unnamed: 0_level_1")]
df = df.drop(("Unnamed: 0_level_0","Unnamed: 0_level_1"), axis=1)

#visualization of the data structure
display(df)

# Daten Visualisierung
In den eingelesenen Daten sind jedem Indent jeweils zwei x- und y-Koordinaten zugeordnet: **„absolut“** und **„real“**. Der vor der Messung festgelegte Abstand zwischen den Indents beträgt 4 µm. Das nachträglich durchgeführte Mapping mit dem Rasterkraftmikroskop zeigt jedoch, dass dieser Abstand – insbesondere bei kleinen Schrittweiten – **deutlich von der eingestellten Schrittweite abweichen kann**.

Die **„absoluten“ Koordinaten** entsprechen den idealisierten Positionen aus den Nanoindentierungsdaten, basierend auf der geplanten Schrittweite. Die **„realen“ Koordinaten** hingegen wurden mithilfe der Rasterkraftmikroskopie (AFM) nachträglich bestimmt und spiegeln die tatsächliche Position der Indents wider.

Wenn man mehrere Datensätze überlagern möchte – wie in diesem Fall die Nanoindentierungsdaten mit den AFM-Daten (oder auch mit anderen Methoden wie z. B. EBSD) – ist es notwendig, die **realen Indentpositionen nachträglich in den Datensatz zu integrieren**. Durch Bildverarbeitung und eine geeignete Skalierung (z. B. auf eine Bildgröße von 50 µm × 50 µm) können die Indentzentren bestimmt werden.

Im vorliegenden Beispiel wurde dieser Schritt bereits mit der Software **ImageJ** durchgeführt, um Zeit zu sparen.
## Code: Erstellen eins Härte- und E-Modulu Mappings
Erstellen Sie mit dem unten liegenden Code für die Härte, das E-Modul jeweils ein Mapping. Passen Sie hier für gegebenenfalls die im Code hervorgehobenen Parameter an.


In [ ]:
#=== change parameter from here ===#
#marker options
marker_shape = "s" #<-- "s" for squares, ">" for triangles
marker_size = 85 # setting the size of the markers
marker_tranparency = 0.75 # setting the transparency of the markers

#choosing between ideal indent position and real indent positions
indent_position = "real" # <-- "real" or "ideal"

#change the colormap of the plots
colormap = "crest_r" #<-- "viridis", "cividis", "inferno" ,"magma", "plasma", "rocket", "flare", "crest", "copper"
#=========================#

#=== dont change paramter from here ===#
#image size
image_width_cm = 20 #change value to alter the image size
image_height_cm = 20 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2.5,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining x axis

#creating a figure object
fig, ax = plt.subplots(nrows = 1,
                       ncols = 3, #3 images next to each other
                       sharey=True)


fig.set_dpi(600) # increasing the resolution of the plot
fig.set_size_inches(image_width_cm/2.5,image_height_cm/2.54) #calcuating image size

# hardness plot 
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = ("HARDNESS GPa","mean"),
                ax = ax[0], #left plot
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,as_cmap = True), # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
               )

if indent_position == "real":
    ax[0].imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

#modulus plot
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = ("MODULUS GPa","mean"),
                ax = ax[1], #middle plot
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,as_cmap = True), # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
                )

if indent_position == "real":
    ax[1].imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )
#stiffnesplot 
sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = ("S2overP","mean"), 
                ax = ax[2], #right plot
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap,as_cmap = True), # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
                )

if indent_position == "real":
    ax[2].imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

for item in ax:
    item.set_aspect("equal") # ensure äquidistance on x and y axis
    sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = item) # move legend above the corresponding plots


# Principal Component Analysis (PCA)

## Die richtigen Daten auswählen

Mit dem Parameter `n_components` bestimmst du, auf wie viele Hauptkomponenten dein Datensatz reduziert wird. Häufig werden 2 oder 3 Komponenten gewählt, da sich diese gut grafisch darstellen lassen.

Die PCA hilft dabei, Datensätze mit vielen Merkmalen (z. B. 5 Features = 5 Dimensionen) auf weniger Dimensionen zu reduzieren. Dabei bleibt die grundlegende Struktur der Daten erhalten – also wie die Messpunkte zueinander in Beziehung stehen. Ziel ist es, die wichtigsten Muster in den Daten sichtbar zu machen und gleichzeitig unwichtige oder redundante Informationen zu reduzieren.

## Wie man PCA-Ergebnisse interpretiert

Die Principal Component Analysis (PCA) hilft dabei, komplexe Datensätze mit vielen Merkmalen auf wenige Hauptkomponenten zu reduzieren. Dadurch lassen sich Muster und Strukturen in den Daten leichter erkennen und visualisieren.

Ein zentraler Anhaltspunkt ist der **Anteil der erklärten Varianz**. Er zeigt, wie viel Information jede Hauptkomponente aus den ursprünglichen Daten aufnimmt. Wenn zum Beispiel die ersten beiden Komponenten zusammen 85 % der Varianz erklären, bedeutet das, dass sie den Großteil der Datenstruktur abbilden. In diesem Fall reicht es oft aus, nur diese beiden Komponenten für die weitere Analyse oder Visualisierung zu verwenden.

Im **PCA-Plot** wird jeder Datenpunkt (z. B. ein Indent) anhand seiner Position im Raum der Hauptkomponenten dargestellt. Liegen Punkte nah beieinander, sind sie sich in ihren Eigenschaften ähnlich. Gruppen oder Cluster im Plot deuten auf vergleichbares Materialverhalten hin, während Ausreißer auf besondere oder fehlerhafte Messungen hinweisen können.

Die sogenannten **Loadings** geben Auskunft darüber, wie stark jedes ursprüngliche Merkmal zu den Hauptkomponenten beiträgt. Sie helfen dabei zu verstehen, was eine Komponente physikalisch bedeutet. Wenn zum Beispiel die erste Komponente stark von der Härte und dem Elastizitätsmodul beeinflusst wird, beschreibt sie vermutlich die allgemeine Steifigkeit des Materials. Eine andere Komponente könnte vor allem durch die Streuung der Messwerte geprägt sein und damit auf die Homogenität oder Heterogenität des Materials hinweisen.

Durch die Kombination dieser Informationen kannst du besser einschätzen, **welche Merkmale die Struktur deiner Daten dominieren** – und wie du sie gezielt für das Clustering oder die Interpretation nutzen kannst.

## Warum müssen die Daten skaliert werden?

Bevor PCA oder KMeans angewendet werden, ist es wichtig, die Daten zu **skalieren** – also alle Merkmale (Features) auf einen vergleichbaren Wertebereich zu bringen.

Das liegt daran, dass viele Algorithmen (wie PCA und KMeans) auf **Abständen im Merkmalsraum** basieren. Wenn ein Feature (z. B. der Elastizitätsmodul in GPa) viel größere Zahlenwerte hat als ein anderes (z. B. die Härte in GPa), dann **dominiert dieses Feature die Analyse**, obwohl es nicht unbedingt wichtiger ist.

Durch das **Standardisieren** (z. B. mit `StandardScaler`) wird jedes Feature so umgerechnet, dass es den **Mittelwert 0 und die Standardabweichung 1** hat. Dadurch tragen alle Merkmale **gleich stark zur Analyse bei** – unabhängig von ihrer ursprünglichen Einheit oder Größenordnung.

---

## Code: Durchführen der PCA  
Probiere verschiedene Werte für `n_components` aus (z. B. 2 und 3) und erweitere den `features`-DataFrame um zusätzliche Messgrößen, die für das Clustering relevant sein könnten – z. B. die Standardabweichung von Härte, Elastizitätsmodul oder S²/P. Scrolle bei Bedarf nach oben, um dir die Struktur des ursprünglichen DataFrames noch einmal anzusehen.


In [ ]:
import os 
os.environ["OMP_NUM_THREADS"] = "1"
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#=== change parameter ===#
n_components = 2 #<-- number of components 

#selecting important columns for clustering
features = df[[
                ("HARDNESS GPa","mean"),
                ("MODULUS GPa","mean"),
                ("S2overP","mean"),
                ("LoadUnloadAnalysis","hf"),
                ("LoadUnloadAnalysis","Energy_Dissipated")
                ]].copy()
#=========================#

#=== dont change paramter from here ===#

#deleting indents with not data (Indent 15, Indent 67) 
features.dropna(inplace = True)

# scale data very important for correct results!
scaler = StandardScaler() 
X_scaled = scaler.fit_transform(features)

# PCA on scaled data
pca = PCA(n_components=n_components) 
X_pca = pca.fit_transform(X_scaled)

fig = plt.figure(figsize=(8, 6)) 
if X_pca.shape[1] == 3:
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2], c='steelblue', s=40)
    ax.set_xlabel('PC 1')
    ax.set_ylabel('PC 2')
    ax.set_zlabel('PC 3')
    ax.set_title('PCA Scatterplot (3D)')
    plt.show()
    print("Erklärte Varianzanteile:", pca.explained_variance_ratio_)
    loadings = pd.DataFrame(pca.components_.T, columns=['PCA1', 'PCA2',"PCA3"], index=features.columns ) 
    print(loadings)

else:
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c='steelblue', s=40)
    plt.xlabel('PC 1')
    plt.ylabel('PC 2')
    plt.title('PCA Scatterplot (2D)')
    plt.show()
    print("Erklärte Varianzanteile:", pca.explained_variance_ratio_)
    loadings = pd.DataFrame(pca.components_.T, columns=['PC 1', 'PC 2'], index=features.columns ) 
    print(loadings)

# deleting PCA columns in orignial data dataframe if present 
if 'PCA' in df.columns.get_level_values(0):
    df = df.drop(columns='PCA', level=0)

# make dataframe for PCA1 and PCA2 and maybe PCA3 axis
pca_columns = [f'PC {i+1}' for i in range(X_pca.shape[1])]
pca_df = pd.DataFrame(X_pca, columns=pca_columns, index=features.index)

# transform to multi index column ("PCA","PC 1),("PC","PC 2"),...
pca_df.columns = pd.MultiIndex.from_product([['PCA'], pca_columns])

# join pca_df with orignal DataFrame df by Index "Indent Nr"
df = df.join(pca_df)

#disply orignal DataFrame with PCA columns
display(df)

# Clustering mit KMeans

## Was ist KMeans und warum ist es für Nanoindentierung sinnvoll?

KMeans ist ein sogenannter **Clustering-Algorithmus**. Er wird verwendet, um Datenpunkte automatisch in Gruppen (sogenannte *Cluster*) einzuteilen – und zwar **ohne dass man vorher wissen muss, welche Gruppen es gibt**. Das macht KMeans zu einem typischen Verfahren des *unüberwachten Lernens*.

1. Es werden zufällig `k` Clusterzentren initialisiert.
2. Jeder Datenpunkt wird dem nächstgelegenen Zentrum zugewiesen.
3. Die Clusterzentren werden neu berechnet – als Mittelwert aller zugehörigen Punkte.
4. Die Schritte 2 und 3 wiederholen sich, bis sich die Clusterzuweisungen nicht mehr ändern oder ein Abbruchkriterium erreicht ist.

In der Nanoindentierung misst man an vielen Punkten mechanische Eigenschaften wie **Härte**, **Elastizitätsmodul** oder **S²/P**. Diese Werte können sich je nach Materialphase, Mikrostruktur oder Oberflächenzustand unterscheiden. KMeans hilft dabei, **ähnliche Indents zu gruppieren**, sodass man z. B. verschiedene Gefügezustände oder Schichten im Material erkennen kann – **rein datengetrieben**, ohne dass man vorher wissen muss, wo sich diese befinden.

---

## Vorbereitung: Wie viele Cluster sind sinnvoll?

### Bewertung der Clusterqualität: Inertia & Silhouette Score

Beim Clustering mit KMeans stellt sich oft die Frage: **Wie viele Cluster sind sinnvoll?** Da KMeans ein unüberwachtes Verfahren ist, gibt es keine „richtige“ Anzahl an Clustern – man muss sie aus den Daten herausfinden. Zwei wichtige Kennzahlen helfen dabei:

#### Inertia (Trägheit)

Die Inertia misst, **wie weit die Datenpunkte im Durchschnitt von ihrem jeweiligen Clusterzentrum entfernt sind**. Je kleiner die Inertia, desto kompakter sind die Cluster.

- **Interpretation:**  
  Eine niedrige Inertia bedeutet, dass die Datenpunkte gut zu ihren Clustern passen. Allerdings sinkt die Inertia immer, wenn man mehr Cluster hinzufügt – auch dann, wenn es keinen echten Mehrwert bringt. Deshalb reicht die Inertia allein nicht aus, um die optimale Clusteranzahl zu bestimmen.

- **Elbow-Methode:**  
  Trägt man die Inertia gegen die Anzahl der Cluster auf, entsteht oft eine Kurve mit einem „Knick“ (engl. *elbow*). Dieser Knick zeigt, **ab wann zusätzliche Cluster nur noch wenig Verbesserung bringen**. Der Punkt des Knicks ist ein guter Kandidat für die optimale Clusteranzahl.

#### Silhouette Score

Der Silhouette Score bewertet, **wie gut ein Punkt zu seinem eigenen Cluster passt – im Vergleich zu anderen Clustern**. Er liegt zwischen –1 und +1:

- **Nahe +1:** Punkt ist gut im eigenen Cluster platziert und weit von anderen Clustern entfernt → gute Trennung  
- **Nahe 0:** Punkt liegt zwischen zwei Clustern → keine klare Zuordnung  
- **Negativ:** Punkt ist vermutlich im falschen Cluster → schlechte Struktur

- **Interpretation:**  
  Ein hoher Silhouette Score spricht für eine sinnvolle Clusterstruktur. Im Gegensatz zur Inertia kann der Score auch wieder sinken, wenn zu viele Cluster gewählt werden – das macht ihn besonders nützlich zur Bewertung.

---

## Code: Bestimmung der geeigneten Anazhl an Cluster

Untersuchen Sie, **wie viele Cluster für die KMeans-Analyse am besten geeignet sind**. Nutzen Sie dazu die Visualisierung der Inertia und des Silhouette Scores.  
Gehen Sie bei Bedarf einen Schritt zurück und **passen Sie die PCA oder das Feature-Set an**, um zu prüfen, wie sich die Scores verändern.


In [ ]:
import os 
os.environ["OMP_NUM_THREADS"] = "1"
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
"""
Computes and plots Silhouette Score and Inertia (Elbow Plot) for different numbers of clusters.
    
Parameters:
- df: DataFrame containing the data and possibly PCA columns
- features_source: 'PCA' or 'features' → determines which data basis to use
- cluster_range: range of cluster numbers to test (e.g., range(2, 10))
 """

#=== Change from here ===#
features_source = "PCA" #<-- "PCA" or "features"
max_number_cluster = 10
#========================#

#=== Dont change from here ===#
silhouette_scores = []
inertias = []
ks = []

# Select data basis
if features_source == 'PCA':
    X = df['PCA'].dropna()
elif features_source == 'features':
    X = features.copy()
else:
    raise ValueError("features_source must be 'PCA' or 'features'.")

for k in range(2,max_number_cluster):
    kmeans = KMeans(n_clusters=k, n_init='auto', random_state=42)
    labels = kmeans.fit_predict(X)
    inertia = kmeans.inertia_
    inertias.append(inertia)
    ks.append(k)

# Silhouette Score is only defined for k > 1
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    print(f"k = {k}, Inertia = {inertia:.2f}, Silhouette Score = {score:.3f}")

# Plotting
fig, ax1 = plt.subplots(figsize=(9, 5))

color1 = 'tab:blue'
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('Inertia (Elbow)', color=color1)
ax1.plot(ks, inertias, marker='o', color=color1, label='Inertia')
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()  # second y-axis for Silhouette Score
color2 = 'tab:green'
ax2.set_ylabel('Silhouette Score', color=color2)
ax2.plot(ks, silhouette_scores, marker='s', linestyle='--', color=color2, label='Silhouette Score')
ax2.tick_params(axis='y', labelcolor=color2)

plt.title('KMeans: Elbow Plot & Silhouette Score')
fig.tight_layout()
plt.grid(True)
plt.show()


## Code: KMeans Clustering

Nutzen Sie die zuvor berechneten Scores, um eine fundierte Entscheidung über die Clusteranzahl zu treffen.  
Passen Sie bei Bedarf das Feature-Set oder die Anzahl der PCA-Komponenten an (_springen Sie hierfür zur PCA-Analyse zurück und änderen Sie die entsprechenden Werte. Führen Sie von dort an den Code der Reihe nach erneut bis hier hin aus_) und beobachten Sie, wie sich die Clusterqualität verändert. Im Vergleich zur vorherigen durchgeführten Cluster-Analyse ist nun die Anzahl der Cluster von Ihnen festgelegt und dem original Dataframe wird eine Spalte hinzugefügt, die jedem Indent in eine dieser Cluster einteilt.


In [ ]:
import os 
os.environ["OMP_NUM_THREADS"] = "1"
from sklearn.cluster import KMeans

# === change parameter from here ===
use_pca = True  # True = using PCA axis for Clustering, False = using orignal feature set 
n_clusters = 2  # number of clusters
#=========================#

#=== dont change paramter from here ===#

if use_pca:
    # extract PCA columns from MultiIndex
    X_kmeans = df['PCA'].dropna()
else:
    # using the feature set defind early as "features = ..." 
    X_kmeans = features.copy()

# apllying index columns to the KMeans DataFrame
X_kmeans = X_kmeans.copy()
X_kmeans.index.name = df.index.name

# KMeans calculations 
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=42)
labels = kmeans.fit_predict(X_kmeans)

# using strings for labelings instaed of numbers
label_names = [f"Gruppe {i+1}" for i in labels]
labels_df = pd.DataFrame(label_names, index=X_kmeans.index)

# creating MultiIndex colum name for joining labels_df with df
labels_df.columns = pd.MultiIndex.from_tuples([('KMeans', 'Label')])

# deleting KMeans grouping if it already exists
if ('KMeans', 'Label') in df.columns:
    df = df.drop(columns=('KMeans', 'Label'))

# join KMeans Label_DataFrame with the original Dataframe df
df = df.join(labels_df)

display(df)

# Clustering mit Gaussian Mixture Models (GMM)

## Was ist GMM?

Gaussian Mixture Models (GMM) sind eine probabilistische Erweiterung des KMeans-Algorithmus. Anstatt Datenpunkte strikt einem Cluster zuzuordnen, geht GMM davon aus, dass die Daten aus einer **Mischung mehrerer mehrdimensionaler Normalverteilungen** stammen. Jeder Datenpunkt erhält dabei **Wahrscheinlichkeiten**, mit denen er zu den einzelnen Clustern gehört – das nennt man **weiches Clustering**.

Diese Flexibilität erlaubt es GMM, **Cluster mit unterschiedlicher Form, Größe und Orientierung** zu modellieren – im Gegensatz zu KMeans, das nur kugelförmige, gleich große Cluster erkennt.

Der Algorithmus nutzt das **Expectation-Maximization (EM)**‑Verfahren und läuft iterativ ab:

1. **Initialisierung**  
   Es werden `k` Gaußverteilungen mit Startparametern (Mittelwert, Kovarianzmatrix, Mischungsgewicht) gesetzt.

2. **E‑Schritt (Expectation)**  
   Für jeden Datenpunkt wird berechnet, **mit welcher Wahrscheinlichkeit** er zu jedem Cluster gehört.

3. **M‑Schritt (Maximization)**  
   Die Parameter der Gaußverteilungen werden aktualisiert – basierend auf den berechneten Wahrscheinlichkeiten.

4. **Wiederholung**  
   Die Schritte 2 und 3 werden wiederholt, bis sich die Parameter kaum noch ändern oder ein Abbruchkriterium erreicht ist.

GMM erlaubt **überlappende Cluster**, unterschiedliche Clusterformen und ist flexibler als KMeans, da es nicht von kugelförmigen Clustern ausgeht.

---

## Bewertung mit dem Log-Likelihood

Da GMM auf Wahrscheinlichkeiten basiert, wird die Modellgüte nicht über Abstände (wie bei KMeans), sondern über den **Log-Likelihood** bewertet. Dieser gibt an, **wie wahrscheinlich es ist, dass das Modell die beobachteten Daten erzeugt hat**.

- **Je höher der Log-Likelihood, desto besser passt das Modell zu den Daten.**
- Um die Ergebnisse besser mit KMeans vergleichen zu können, wird häufig der **negative Log-Likelihood** geplottet – so entsteht ein ähnlicher „Elbow-Plot“, bei dem man nach einem Knick sucht.

---

## Wann ist GMM sinnvoll?

GMM ist besonders hilfreich, wenn:

- **Cluster unterschiedliche Formen oder Dichten aufweisen**
- **Cluster sich überlappen** und eine harte Zuweisung (wie bei KMeans) zu ungenau wäre
- **Wahrscheinlichkeiten für die Clusterzugehörigkeit** von Interesse sind (z. B. zur Identifikation unsicherer oder gemischter Bereiche)
- **Materialphasen oder Gefügezustände fließend ineinander übergehen**, statt klar getrennt zu sein

---

## Code: Bestimmung der geeigneten Anzahl an Cluster
Füren Sie den unten stehenden Code-Abschnitt aus und analysieren sie welche Cluster-Anzahl für für den GMM-Algorithmus auf Grund des zuvor definierten Feature-Sets und der PCA Analyse am sinnvollsten ist. Passen sie gegebenenfalls die PCA-Analyse an. Gehen Sie wie beim KMeans-Algorithmus vor.

In [ ]:
# GMM Clustering
import os 
os.environ["OMP_NUM_THREADS"] = "1"
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.mixture import GaussianMixture

"""
Computes and plots Silhouette Score and Negative Log-Likelihood (as Elbow Plot) 
for different numbers of clusters using Gaussian Mixture Models (GMM).

Parameters:
- df: DataFrame containing the data and possibly PCA columns
- features_source: 'PCA' or 'features' → determines which data basis to use
- max_number_cluster: maximum number of clusters to test (e.g., 10)
"""

# === Change from here === #
features_source = "PCA"  # <-- "PCA" or "features"
max_number_cluster = 10
# ======================== #

# === Don't change from here === #
silhouette_scores = []
neg_log_likelihoods = []
ks = []

# Select data basis
if features_source == 'PCA':
    X = df['PCA'].dropna()
elif features_source == 'features':
    X = features.copy()
else:
    raise ValueError("features_source must be 'PCA' or 'features'.")

for k in range(2, max_number_cluster):
    gmm = GaussianMixture(n_components=k, random_state=42, n_init=5)
    gmm.fit(X)
    labels = gmm.predict(X)
    
    # Log-likelihood (higher is better, so we invert it for Elbow-like plot)
    neg_ll = -gmm.score(X) * len(X)
    neg_log_likelihoods.append(neg_ll)
    ks.append(k)

    # Silhouette Score
    score = silhouette_score(X, labels)
    silhouette_scores.append(score)
    print(f"k = {k}, -LogLikelihood = {neg_ll:.2f}, Silhouette Score = {score:.3f}")

# Plotting
fig, ax1 = plt.subplots(figsize=(9, 5))

color1 = 'tab:purple'
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('− Log-Likelihood (Elbow)', color=color1)
ax1.plot(ks, neg_log_likelihoods, marker='o', color=color1, label='− Log-Likelihood')
ax1.tick_params(axis='y', labelcolor=color1)

ax2 = ax1.twinx()  # second y-axis for Silhouette Score
color2 = 'tab:green'
ax2.set_ylabel('Silhouette Score', color=color2)
ax2.plot(ks, silhouette_scores, marker='s', linestyle='--', color=color2, label='Silhouette Score')
ax2.tick_params(axis='y', labelcolor=color2)

plt.title('GMM: Log-Likelihood & Silhouette Score')
fig.tight_layout()
plt.grid(True)
plt.show()


## Code: GMM Clustering

Nutzen Sie die zuvor berechneten Scores (z. B. BIC, AIC oder Silhouette), um eine fundierte Entscheidung über die optimale Anzahl an Clustern für das GMM zu treffen.

Falls notwendig:

- passen Sie das **Feature‑Set** an oder  
- verändern Sie die Anzahl der **PCA‑Komponenten**

(gehen Sie hierfür zur PCA‑Analyse zurück, ändern Sie die entsprechenden Werte und führen Sie den Code erneut bis zu diesem Punkt aus).

Beobachten Sie, wie sich die Clusterqualität verändert.

Im Gegensatz zur vorherigen Clusteranalyse legen Sie nun die Anzahl der Cluster selbst fest.  
Dem ursprünglichen DataFrame wird anschließend eine neue Spalte hinzugefügt, die jedem Datenpunkt ein GMM‑Cluster zuweist.


In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

from sklearn.mixture import GaussianMixture

# === change parameter from here ===
use_pca = True   # True = using PCA axis for Clustering, False = using original feature set 
n_clusters = 2   # number of mixture components
gmm_n_init = 10   # number of initializations for GMM
# ======================== #

# === don't change parameter from here === #

if use_pca:
    # extract PCA columns from MultiIndex
    X_gmm = df['PCA'].dropna()
else:
    # using the feature set defined earlier as "features = ..."
    X_gmm = features.copy()

# apply index columns to the GMM DataFrame
X_gmm = X_gmm.copy()
X_gmm.index.name = df.index.name

# GMM fitting
gmm = GaussianMixture(n_components=n_clusters, random_state=42, n_init=gmm_n_init)
gmm.fit(X_gmm)
labels = gmm.predict(X_gmm)
probs = gmm.predict_proba(X_gmm)  # shape: (n_samples, n_components)

# using strings for labelings instead of numbers
label_names = [f"Gruppe {i+1}" for i in labels]
labels_df = pd.DataFrame(label_names, index=X_gmm.index)

# create MultiIndex column name for joining labels_df with df
labels_df.columns = pd.MultiIndex.from_tuples([('GMM', 'Label')])

# create a DataFrame for component probabilities with MultiIndex columns
prob_cols = [ ( 'GMM', f'comp_{i+1}' ) for i in range(n_clusters) ]
prob_df = pd.DataFrame(probs, index=X_gmm.index, columns=pd.MultiIndex.from_tuples(prob_cols))

# delete existing GMM grouping/prob columns if they already exist
cols_to_drop = []
if ('GMM', 'Label') in df.columns:
    cols_to_drop.append(('GMM', 'Label'))
for col in prob_df.columns:
    if col in df.columns:
        cols_to_drop.append(col)
if cols_to_drop:
    df = df.drop(columns=cols_to_drop)

# join label and probability DataFrames with the original DataFrame df
df = df.join(labels_df)
df = df.join(prob_df)

display(df)


# Cluster Mapping
## Code: Plotting der Clusterlabels auf die x‑ und y‑Koordinaten

Verwenden Sie den untenstehenden Code, um Cluster‑Mappings zu erstellen.  
Im Gegensatz zum Beginn der Analyse können nun nicht mehr nur einzelne mechanische Werte wie E‑Modul oder Härte visualisiert werden, sondern auch die **Clusterlabels** selbst.

Dadurch entsteht eine Karte, die Bereiche mit ähnlichen Eigenschaften farblich hervorhebt und so räumliche Muster oder Materialzonen sichtbar macht.


In [ ]:
#=== change parameter from here ===#
#marker options
marker_shape = "s" #<-- "s" for squares, ">" for triangles
marker_size = 85 # setting the size of the markers
marker_tranparency = 0.75 # setting the transparency of the markers

#choosing between ideal indent position and real indent positions
indent_position = "real" # <-- "real" or "ideal"

#change the colormap of the plots
colormap = "tab10" #<-- "tab10", "tab20", "colorblind" 

hue = ("GMM","Label")
#=========================#

#=== dont change paramter from here ===#
#image size
image_width_cm = 6 #change value to alter the image size
image_height_cm = 6 # change value to alter the image size

#Atomic Force Microscopy Mapping of Indentationmapping
background_image = plt.imread("Data/Images/BackGround.png")
dx,dy = -2.5,-3.2 #parameter for adjusting the background image
range_um = 50 # parameter for adjusting the background image

# automatical column selection for indent position
if indent_position == "real":
    x = ("x","real") # defining x axis
    y = ("y","real") # defining y axis
elif indent_position =="ideal":
    x = ("x","absolut") # defining x axis
    y = ("y","absolut") # defining x axis

#creating a figure object
fig, ax = plt.subplots(nrows = 1,
                       ncols = 1, 
                       sharey=False)


fig.set_dpi(600) # increasing the resolution of the plot
fig.set_size_inches(image_width_cm/2.5,image_height_cm/2.54) #calcuating image size


sns.scatterplot(data = df,
                x = x,
                y = y,
                hue = hue,
                ax = ax,
                marker = marker_shape, #square marker
                palette = sns.color_palette(colormap)[2:], # defining the color plaette
                s = marker_size, # marker size
                edgecolor = None, # remove the outline of the markers
                alpha = marker_tranparency , # transparence of the markers infill
               )

if indent_position == "real":
    ax.imshow(background_image,
                 extent=[dx,range_um+dx,dy,range_um+dy],
                 aspect='equal',
                 zorder=-1,
                 origin = "upper"
                )

ax.set_aspect("equal") # ensure äquidistance on x and y axis
sns.move_legend(loc = "lower center", bbox_to_anchor = (0.5,1), obj = ax) # move legend above the corresponding plots


# Datenaufbereitung

Betrachtet man den Silhouette Score und den Elbow-Plot für die GMM‑ und KMeans‑Analyse, fällt auf, dass für die meisten Feature-Sets oder PCA‑Achsen weder bei GMM noch bei KMeans ein klar erkennbarer Knick im Elbow-Plot vorhanden ist. Zudem liegt der Silhouette Score in den meisten Fällen unterhalb von 0.4. Dies weist darauf hin, dass die Trennschärfe zwischen den einzelnen Clustern sehr gering ist.

Ursachen hierfür können sein:

- ein zu kleiner Datensatz (zu wenige Indents)  
- eine zu große Streuung innerhalb der Daten  

Welche Variante — mehr Daten mit höherer Streuung oder weniger Daten mit geringerer Streuung — für einen Datensatz vorteilhafter ist, muss empirisch untersucht werden.

Für diese Bewertung ist ein gutes Verständnis der vorliegenden Daten entscheidend:

- Wie wurden die Daten erhoben?  
- Welche physikalische Bedeutung besitzen sie?

Laden Sie die Datei **„DataWithAdditional_Infos.csv“** und verschaffen Sie sich einen Überblick über den Datensatz.  
Alle Mittelwerte („mean“) und Standardabweichungen („std“) wurden über einen Eindringtiefenbereich von **200 bis 400 nm** berechnet. Die zugehörigen Kraft‑Weg‑, E‑Modul‑Weg‑ und Härte‑Weg‑Datensätze sind aufgrund der Datenmenge nicht Teil dieser Übung.

Über denselben Eindringtiefenbereich (200–400 nm) wurde ein linearer Fit für die Härte‑Weg‑, E‑Modul‑Weg‑ und Steifigkeit‑Weg‑Datensätze durchgeführt. Dieser Fit ermöglicht es zu beurteilen, ob die Werte (E‑Modul, Härte) über die betrachtete Eindringtiefe steigen, fallen oder konstant bleiben.  
Ab einer ausreichend großen Eindringtiefe sollten die Werte weitgehend konstant sein. Ist dies nicht der Fall, kann dies darauf hindeuten, dass:

- ein Indent zwei Phasen gleichzeitig getroffen hat oder  
- sich im Bereich 0–400 nm mehrere Phasen untereinander befinden.

Solche Indents verwischen die Clustergrenzen und können die Clusterqualität deutlich verschlechtern. Daher kann es sinnvoll sein, diese Indents aus dem Datensatz zu entfernen.

---

## Code: Datenbearbeitung

Führen Sie mithilfe des untenstehenden Codes eine Datenfilterung durch und entfernen Sie alle Indents, deren E‑Modul im Auswertebereich der Eindringtiefe (200–400 nm) um mehr als **10 %** schwankt.  
Die entsprechende Spalte im Datensatz lautet: **("MODULUS GPa", "Slope%")**.

Lassen Sie den Code bei Bedarf mehrfach laufen und verändern Sie jeweils:

- den Parameter `value_to_filter`  
- die Werte für `min_value` und `max_value`

um nach mehreren Kriterien zu filtern.

Führen Sie anschließend alle Codeabschnitte ab der **Principal Component Analysis (PCA)** erneut aus und überprüfen Sie, ob sich der Elbow‑Plot oder der Silhouette Score für KMeans und GMM verbessert hat.


In [ ]:
value_to_filter = ("MODULUS GPa","Slope%")
min_value = -5
max_value = 5

df = df[(df[value_to_filter] <= max_value) &(df[value_to_filter] >= min_value)]

#only to show that the length of the DataFrame has changed
display(df[("MODULUS GPa","Slope%")])

# Weitere Cluster‑Darstellungen

## Code: Plotten unterschiedlicher Cluster‑Darstellungen

Verwenden Sie den untenstehenden Code, um verschiedene Darstellungen der Cluster im Feature‑Space oder im PCA‑Space zu visualisieren.  
Wählen Sie hierfür für **x**, **y** und **z** die jeweiligen Spalten aus, die gegeneinander aufgetragen werden sollen.  
Über den Parameter **hue** legen Sie fest, nach welchem Kriterium die Datenpunkte eingefärbt werden.

Besonders geeignet sind die Clusterlabels  
- **("GMM", "Label")** oder  
- **("KMeans", "Label")**,  

aber auch kontinuierliche (nicht‑kategorische) Spalten können verwendet werden, um zusätzliche datenabhängige Zusammenhänge sichtbar zu machen.

Für eine **2D‑Darstellung** setzen Sie den Parameter:
z = None


In [ ]:

#=== change from here ===#
plot_dataframe(df, #<-- keep it 
               x=('MODULUS GPa','mean'), # choose x-axis
               y=("HARDNESS GPa",'mean'), # choose y-axis
               z = ("S2overP","mean"), # choose z-axis or set it to None for 2D plot
               hue=('KMeans','Label') # choosing colorations 
              ) 
#=== dont change from here ===#

